In [1]:
import csv
import urllib.request
import xml.etree.ElementTree as ET
import time
from functools import wraps

import pandas as pd
import math

import csv
import urllib3
import xml.etree.ElementTree as ET
import time
from functools import wraps

from bs4 import BeautifulSoup as soup
import re

from tqdm import tqdm

In [2]:
http = urllib3.PoolManager()

def urlopen_with_retry(url):
    for _ in range(4):
        try:
            r = http.request('GET', url)
            
            break  # success
        except Exception as e:
            print('sleeping')
            time.sleep(3)
            raise  # propagate non-timeout errors
    else:  # all ntries failed
        raise   # re-raise the last timeout error
        # use page here
    return r


def htmlRequest(urlTop, page, urlBottom=''):
#     print(urlTop + page + urlBottom)
    file = urlopen_with_retry(urlTop + page + urlBottom)
    
    return soup(file.data, 'lxml')

# Pull Top Games

In [3]:
def doc2links(doc):
    out_lst = []
    doc_tbl = doc.find(id='collectionitems')
    for x in doc_tbl.find_all(class_='collection_objectname'):
        out_lst.append(x.find('a')['href'])
    
    return out_lst

game_url_lst = []

for i in range(1, 188):
    game_url_lst = game_url_lst + doc2links(htmlRequest('https://boardgamegeek.com/browse/boardgame/page/', str(i)))
    
len(game_url_lst)

18700

In [4]:
game_url_lst[:3]

['/boardgame/174430/gloomhaven',
 '/boardgame/161936/pandemic-legacy-season-1',
 '/boardgame/167791/terraforming-mars']

## Extract IDs

In [5]:
p = re.compile('(?<=/boardgame/).*(?=/)')

game_ids = []
for x in game_url_lst:
    try:
        game_ids.append(p.search(x).group())
    except:
        print(x)
        
print(len(game_ids))
game_ids[:3]

/boardgameexpansion/79/medfront
/boardgameexpansion/80/volgafront
/boardgameexpansion/81/eurofront
/boardgameexpansion/158/elfengold
18696


['174430', '161936', '167791']

# Get Game Data

In [6]:
def attrs2lst(item, attr):
    out_lst = []
    for x in item.find_all(attr):
        out_lst.append(x.string)
    return out_lst

def find_name(game):
    for x in game.find_all('name'):
        try:
            if x['primary'] == 'true':
                return x.string
        except:
            pass
        
    return None

def xml2df(xml):
    out_dict = {}
    for game in xml.find_all('boardgame'):
        game_id = game['objectid']
        out_dict[game_id] = {}
        try:
            out_dict[game_id]['name'] = find_name(game)
        except:
            out_dict[game_id]['name'] = None
        try:
            out_dict[game_id]['description'] = game.find('description').string
        except:
            out_dict[game_id]['description'] = None

        out_dict[game_id]['boardgamecategorys'] = attrs2lst(game, 'boardgamecategory')
        out_dict[game_id]['boardgamemechanics'] = attrs2lst(game, 'boardgamemechanic')

    return pd.DataFrame.from_dict(out_dict, orient='index')

In [7]:
n = 500
df_lst = []

for i in tqdm(range(n, len(game_ids)+n, n)):
    s = i - n
    if i > len(game_ids):
        i = len(game_ids)
    xml = htmlRequest('https://www.boardgamegeek.com/xmlapi/boardgame/', ','.join(game_ids[s:i]))
    df_lst.append(xml2df(xml))

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [07:48<00:00, 12.34s/it]


In [8]:
df_lst[0]

,name,description,boardgamecategorys,boardgamemechanics
174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Retrieval, Campaign / Battle Card Driv..."
161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem..."
167791,Terraforming Mars,"In the 2400s, mankind begins to terraform the ...","[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem..."
224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,"[Economic, Industry / Manufacturing, Transport...","[Connections, Hand Management, Income, Loans, ..."
182028,Through the Ages: A New Story of Civilization,Through the Ages: A New Story of Civilization ...,"[Card Game, Civilization, Economic]","[Action Points, Auction/Bidding, Auction: Dutc..."
...,...,...,...,...
253344,Cthulhu: Death May Die,"In Cthulhu: Death May Die, inspired by the wri...","[Fantasy, Fighting, Horror]","[Action Points, Cooperative Game, Dice Rolling..."
10547,Betrayal at House on the Hill,From the press release:<br/><br/>Betrayal at H...,"[Adventure, Exploration, Horror, Miniatures]","[Dice Rolling, Map Addition, Modular Board, Pl..."
170561,Valeria: Card Kingdoms,The land of Valeria is under siege by hordes o...,"[Card Game, Dice, Fantasy]","[Card Drafting, Deck, Bag, and Pool Building, ..."
586,Up Front,Only once in a great while does a game dare to...,"[Card Game, Wargame, World War II]","[Campaign / Battle Card Driven, Hand Managemen..."


In [9]:
df = pd.concat(df_lst)

print(df.shape)
df.head()

(18740, 4)


,name,description,boardgamecategorys,boardgamemechanics
174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Retrieval, Campaign / Battle Card Driv..."
161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem..."
167791,Terraforming Mars,"In the 2400s, mankind begins to terraform the ...","[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem..."
224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,"[Economic, Industry / Manufacturing, Transport...","[Connections, Hand Management, Income, Loans, ..."
182028,Through the Ages: A New Story of Civilization,Through the Ages: A New Story of Civilization ...,"[Card Game, Civilization, Economic]","[Action Points, Auction/Bidding, Auction: Dutc..."


In [10]:
df.to_csv('bgg_data.csv')